## Faiss

Facebook AI Similarity Search (Faiss) is a library for efficient similarity search and clustering of dense vectors. It contains algorithms that search in sets of vectors of any size, up to ones that possibly do not fit in RAM, It also contains supporting code for evaluation and parameter tunning.

In [24]:
from langchain_community.document_loaders import TextLoader
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_huggingface.embeddings import HuggingFaceEmbeddings
from langchain_text_splitters import CharacterTextSplitter
import os
from dotenv import load_dotenv

load_dotenv()

os.environ["HF_API_TOKEN"] = os.getenv("HF_API_TOKEN")

loader = TextLoader("speech.txt")

docunets = loader.load()

text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=30)

docs = text_splitter.split_documents(docunets)

In [25]:
docs

[Document(metadata={'source': 'speech.txt'}, page_content='The world must be made safe for democracy. Its peace must be planted upon the tested foundations of political liberty. We have no selfish ends to serve. We desire no conquest, no dominion. We seek no indemnities for ourselves, no material compensation for the sacrifices we shall freely make. We are but one of the champions of the rights of mankind. We shall be satisfied when those rights have been made as secure as the faith and the freedom of nations can make them.\n\nJust because we fight without rancor and without selfish object, seeking nothing for ourselves but what we shall wish to share with all free peoples, we shall, I feel confident, conduct our operations as belligerents without passion and ourselves observe with proud punctilio the principles of right and of fair play we profess to be fighting for.\n\n…'),
 Document(metadata={'source': 'speech.txt'}, page_content='…\n\nIt will be all the easier for us to conduct our

In [26]:
embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

In [27]:
db = FAISS.from_documents(docs, embeddings)
db

In [28]:
### querying

# query = "What does the speaker believe is the main reason the United State should enbter the war?"
query = "how does the speaker describes the desired outcome of the war?"

In [29]:
result = db.similarity_search(query)

In [30]:
result

[Document(id='30262e7b-8e7c-4a14-ba48-5d117992f25e', metadata={'source': 'speech.txt'}, page_content='It is a distressing and oppressive duty, gentlemen of the Congress, which I have performed in thus addressing you. There are, it may be, many months of fiery trial and sacrifice ahead of us. It is a fearful thing to lead this great peaceful people into war, into the most terrible and disastrous of all wars, civilization itself seeming to be in the balance. But the right is more precious than peace, and we shall fight for the things which we have always carried nearest our hearts—for democracy, for the right of those who submit to authority to have a voice in their own governments, for the rights and liberties of small nations, for a universal dominion of right by such a concert of free peoples as shall bring peace and safety to all nations and make the world itself at last free.'),
 Document(id='51f195da-b135-42bb-9402-47c09cb1c112', metadata={'source': 'speech.txt'}, page_content='The

## As Retriver

We can also convert the vectorstore into a Retriver class. This allows us to easily use it in other LangChain methods, which largely work with retrivers

In [32]:
retriever = db.as_retriever()
docs = retriever.invoke(query)
docs[0].page_content

'It is a distressing and oppressive duty, gentlemen of the Congress, which I have performed in thus addressing you. There are, it may be, many months of fiery trial and sacrifice ahead of us. It is a fearful thing to lead this great peaceful people into war, into the most terrible and disastrous of all wars, civilization itself seeming to be in the balance. But the right is more precious than peace, and we shall fight for the things which we have always carried nearest our hearts—for democracy, for the right of those who submit to authority to have a voice in their own governments, for the rights and liberties of small nations, for a universal dominion of right by such a concert of free peoples as shall bring peace and safety to all nations and make the world itself at last free.'

## Similarity Search with Score

There are some FAISS specific methods. One of them is similarity_search_with_score , which allows you to return not only the documents but also the distance score of the query to them. The returned distance score is L2 distance (manhatten distance) . Therefore , a lower score is better

In [34]:
docs_and_score = db.similarity_search_with_score(query)

docs_and_score

[(Document(id='30262e7b-8e7c-4a14-ba48-5d117992f25e', metadata={'source': 'speech.txt'}, page_content='It is a distressing and oppressive duty, gentlemen of the Congress, which I have performed in thus addressing you. There are, it may be, many months of fiery trial and sacrifice ahead of us. It is a fearful thing to lead this great peaceful people into war, into the most terrible and disastrous of all wars, civilization itself seeming to be in the balance. But the right is more precious than peace, and we shall fight for the things which we have always carried nearest our hearts—for democracy, for the right of those who submit to authority to have a voice in their own governments, for the rights and liberties of small nations, for a universal dominion of right by such a concert of free peoples as shall bring peace and safety to all nations and make the world itself at last free.'),
  1.2987),
 (Document(id='51f195da-b135-42bb-9402-47c09cb1c112', metadata={'source': 'speech.txt'}, page

In [35]:
embeddings_vector = embeddings.embed_query(query)
embeddings_vector

[-0.032913755625486374,
 0.10170978307723999,
 -0.00011212809477001429,
 -0.019504006952047348,
 -0.045224182307720184,
 0.06250227987766266,
 0.04311902076005936,
 -0.028385551646351814,
 0.026878857985138893,
 0.024963034316897392,
 -0.04218880832195282,
 0.03370293974876404,
 0.018711458891630173,
 -0.01545436680316925,
 -0.0009912289679050446,
 0.003464618930593133,
 0.015736691653728485,
 -0.03418785333633423,
 0.009261156432330608,
 -0.01245136372745037,
 0.09701213985681534,
 0.054180487990379333,
 0.06386708468198776,
 0.010363666340708733,
 0.019825933501124382,
 -0.019199838861823082,
 -0.020216049626469612,
 0.049648795276880264,
 -0.014786963351070881,
 0.03600902482867241,
 0.054893769323825836,
 -0.01741040125489235,
 0.019715070724487305,
 0.003939272370189428,
 0.00608086958527565,
 0.009839949198067188,
 -0.06848147511482239,
 -0.0268461462110281,
 -0.016417721286416054,
 -0.015818411484360695,
 -0.0526374876499176,
 -0.04006645828485489,
 0.014256134629249573,
 0.0425

In [37]:
docs_and_score = db.similarity_search_by_vector(embeddings_vector)

In [38]:
docs_and_score

[Document(id='30262e7b-8e7c-4a14-ba48-5d117992f25e', metadata={'source': 'speech.txt'}, page_content='It is a distressing and oppressive duty, gentlemen of the Congress, which I have performed in thus addressing you. There are, it may be, many months of fiery trial and sacrifice ahead of us. It is a fearful thing to lead this great peaceful people into war, into the most terrible and disastrous of all wars, civilization itself seeming to be in the balance. But the right is more precious than peace, and we shall fight for the things which we have always carried nearest our hearts—for democracy, for the right of those who submit to authority to have a voice in their own governments, for the rights and liberties of small nations, for a universal dominion of right by such a concert of free peoples as shall bring peace and safety to all nations and make the world itself at last free.'),
 Document(id='51f195da-b135-42bb-9402-47c09cb1c112', metadata={'source': 'speech.txt'}, page_content='The

In [40]:
### Saving and Loading
db.save_local("fiass_index")

In [42]:
new_df = FAISS.load_local("fiass_index", embeddings, allow_dangerous_deserialization=True)